In [1]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')


# own libs
import sys
sys.path.append('//home//roman_vm//my_lib//')

from data_load import *

In [2]:
df=get_data("""

select *
from chesscom_dataset


""")

In [3]:
df['datime'] = df['datime'].astype('datetime64[ns]')
df['year']=df['datime'].dt.year
df['month']=df['datime'].dt.month
df['week']=df['datime'].dt.week
df['day']=df['datime'].dt.day
df['weekday']=df['datime'].dt.weekday
df['hour']=df['datime'].dt.hour

In [4]:
df.sample(3)

,level_0,index,target_user,opponent,datime,event,rules,time_class,time_control,rated,result,score,result_reason,opponent_result_reason,color,rating,opponent_rating,base_elo_forec,eco,game_url,eco_url,inserted_at,game_number,last_game_at,minutes_from_last_game,is_new_session,game_id,session_id,game_ss_number,intransigence_rate,opponent_intransigence_rate,clocker_rate,opponent_clocker_rate,target_opponent_clocker_rate,opponent_opponent_clocker_rate,personal_meets_stat,is_weekend,is_night,year,month,week,day,weekday,hour
8938,8938,0,Rosolimo,jahan466,2015-12-17 10:01:30,Live Chess,chess,blitz,300,True,0-1,1.0,win,resigned,b,1542,1593,0.427128,B13,https://www.chess.com/live/game/1388226223,https://www.chess.com/openings/Caro-Kann-Defense-Exchange-Variation-3...cxd5-4.Nf3,2020-07-26 15:29:55.043194,6062,2015-12-17 09:57:51,3.0,0,1,Rosolimo-634,2,0.075,None,0.008,None,0.13,None,1.000000,0,0,2015,12,51,17,3,10
2647,2647,0,Rosolimo,TRUSTemy,2019-03-21 22:06:22,Live Chess,chess,blitz,180+2,True,1/2-1/2,0.5,agreed,agreed,b,1513,1485,0.540208,C10,https://www.chess.com/live/game/3553795231,https://www.chess.com/openings/French-Defense-Rubinstein-Blackburne-Defense-5.Nf3-Ngf6-6.Bd3,2020-07-26 15:29:55.043194,11027,2019-03-21 22:01:56,4.0,0,1,Rosolimo-1483,4,0.075,None,0.008,None,0.13,None,0.666667,0,0,2019,3,12,21,3,22
9269,9269,0,Rosolimo,timurdemiral,2016-01-18 10:54:13,Live Chess,chess,blitz,300,True,1/2-1/2,0.5,insufficient,insufficient,w,1611,1565,0.565815,A53,https://www.chess.com/live/game/1427329293,https://www.chess.com/openings/Old-Indian-Defense-3.Nf3,2020-07-26 15:29:55.043194,6348,2016-01-18 10:48:29,5.0,0,1,Rosolimo-671,5,0.075,None,0.008,None,0.13,None,0.500000,0,0,2016,1,3,18,0,10


In [5]:
len(df)

12281

In [6]:
f_list=[u'event',
       u'rules', u'time_class', u'time_control',  
       u'result_reason', u'opponent_result_reason', 
       u'color', u'rating', u'opponent_rating', u'base_elo_forec', 
       u'eco',
       u'minutes_from_last_game', u'game_ss_number', 
       u'intransigence_rate', u'opponent_intransigence_rate', 
       u'clocker_rate', u'opponent_clocker_rate', 
       u'target_opponent_clocker_rate', u'opponent_opponent_clocker_rate', 
       u'personal_meets_stat', 
       u'is_weekend', u'is_night', 
       u'year', u'month', u'week', u'day', u'weekday', u'hour',
       u'score'
       ]

In [7]:
datatypes=pd.DataFrame(df.dtypes, columns=['type']).reset_index()
f_df=pd.DataFrame(f_list, columns=['is_in_f_lst'])
f_df=f_df.merge(datatypes, 'left', left_on='is_in_f_lst', right_on='index')
f_df=f_df.drop(columns='index')
eho_cols=list(f_df[f_df['type']=='object']['is_in_f_lst'].values)
number_cols=list(f_df[f_df['type']!='object']['is_in_f_lst'].values)

In [8]:
for col in f_df[f_df['type']=='int64']['is_in_f_lst'].values:
    df[col]=df[col].fillna(0)
for col in f_df[f_df['type']=='float64']['is_in_f_lst'].values:
    df[col]=df[col].fillna(0.0)
for col in f_df[f_df['type']=='object']['is_in_f_lst'].values:
    df[col]=df[col].fillna(col+'Unknown')

In [9]:
work_df=df[number_cols]
for col in eho_cols:
    buf_df=pd.get_dummies(df[col])
    work_df=pd.concat([work_df, buf_df], axis=1)

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

for col in number_cols:
    work_df[col]=scaler.fit_transform(work_df[[col]])

In [19]:
work_df.sample(3)

,rating,opponent_rating,base_elo_forec,minutes_from_last_game,game_ss_number,intransigence_rate,clocker_rate,target_opponent_clocker_rate,personal_meets_stat,is_weekend,is_night,year,month,week,day,weekday,hour,score,Live Chess,chess,blitz,bullet,rapid,120+1,180,180+2,300,300+2,300+3,60,60+1,600,600+2,50move,abandoned,agreed,checkmated,insufficient,repetition,resigned,stalemate,timeout,timevsinsufficient,win,50move,abandoned,agreed,checkmated,insufficient,repetition,resigned,stalemate,timeout,timevsinsufficient,win,b,w,A00,A01,A02,A03,A04,A06,A07,A08,A09,A10,A11,A13,A15,A16,A17,A20,A21,A22,A25,A28,A30,A31,A32,A33,A34,A35,A36,A40,A41,A42,A43,A44,A45,A46,A48,A49,A50,A51,A52,A53,A54,A56,A57,A58,A60,A65,A70,A80,A84,A85,B00,B01,B02,B03,B06,B07,B08,B09,B10,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20,B21,B22,B23,B30,B33,B34,B40,B44,B45,B50,B52,B54,C00,C01,C02,C07,C08,C10,C11,C14,C15,C20,C31,C34,C41,C42,C44,C46,C61,C62,C63,C64,C66,C70,C77,D00,D01,D02,D03,D04,D06,D07,D08,D10,D11,D13,D15,D16,D20,D21,D24,D30,D31,D32,D35,D37,D38,D40,D41,D43,D44,D45,D46,D48,D50,D51,D52,D53,D55,D60,D70,D72,D73,D77,D78,D80,D81,D82,D83,D84,D85,D86,D87,D90,D91,D92,D93,D94,D95,D96,D97,D98,E00,E10,E11,E12,E14,E15,E16,E17,E18,E19,E20,E21,E32,E33,E34,E38,E40,E41,E43,E60,E61,E62,E70,E76,E80,E90,E91,E92,E93,E94,Unknown,opponent_intransigence_rateUnknown,opponent_clocker_rateUnknown,opponent_opponent_clocker_rateUnknown
2522,0.584551,0.540163,0.564143,0.457627,0.016949,0.0,0.0,0.0,0.00,0.0,0.0,0.857143,0.181818,0.173077,0.1,0.000000,0.782609,0.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
5955,0.638831,0.596042,0.461935,0.118644,0.016949,0.0,0.0,0.0,0.75,1.0,0.0,0.142857,0.909091,0.884615,0.7,0.833333,0.739130,0.5,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
2288,0.634656,0.570431,0.523477,0.288136,0.084746,0.0,0.0,0.0,0.50,1.0,0.0,0.857143,0.000000,0.019231,0.4,1.000000,0.956522,1.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1


In [20]:
df['datime'].values[0]

numpy.datetime64('2013-09-16T10:14:36.000000000')

In [22]:
hold_point='2019-08-01'
test_point='2019-08-01'

In [30]:
work_df['date']=df['datime']
hold_df=work_df[work_df['date']>=hold_point]
ds=work_df[work_df['date']<hold_point]
ds=ds.drop(columns='date')

X=ds.copy().drop(columns='score')
y=ds['score']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X.values, y.values, test_size=0.25, random_state=42)

In [ ]:
# строит график, считает метрики
def check(df, x, y1, y2, title):
    from sklearn.metrics import mean_squared_error
    print(title)
    df[[x, y1, y2]].set_index(x).plot(title=title)
    
    print('correlation: ', np.corrcoef(df[y1], df[y2])[0][1])
    
    print('stdev: ', mean_squared_error(df[y1], df[y2]))

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train, y_train)
lr_pred=lr.predict(X_test)

print('R^2 train:', lr.score(X_train, y_train))
print('R^2 test:', lr.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(lr.predict(X_test)))
print('MAE train', mean_absolute_error(lr.predict(X_train), y_train))
print('MAE test', mean_absolute_error(lr.predict(X_test), y_test))

# plt.plot(range(len(y_test)), y_test, lr.predict(X_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)

lr_pred=lr.predict(X_test)

print('R^2 train:', lr.score(X_train, y_train))
print('R^2 test:', lr.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(lr.predict(X_test)))
print('MAE train', mean_absolute_error(lr.predict(X_train), y_train))
print('MAE test', mean_absolute_error(lr.predict(X_test), y_test))

# plt.plot(range(len(y_test)), y_test, lr.predict(X_test))

In [ ]:
import xgboost
from sklearn.metrics import mean_absolute_error, mean_squared_error
xgb = xgboost.XGBRegressor()
xgb.fit(X_train, y_train)
print('R^2 train:', xgb.score(X_train, y_train))
print('R^2 test:', xgb.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(xgb.predict(X_test)))
print('MAE train', mean_absolute_error(xgb.predict(X_train), y_train))
print('MAE test', mean_absolute_error(xgb.predict(X_test), y_test))

In [ ]:
xgb_model_abs_testing = list()
xgb_model_abs_training = list()
rng = np.arange(1,151)
for i in rng:
    xgb = xgboost.XGBRegressor(n_estimators=i)
    xgb.fit(X_train, y_train)
    xgb.score(X_test, y_test)
    xgb_model_abs_testing.append(mean_absolute_error(xgb.predict(X_test), y_test))
    xgb_model_abs_training.append(mean_absolute_error(xgb.predict(X_train), y_train))
    
find_df=pd.DataFrame(np.arange(1,151), columns=['n_estimators']).set_index('n_estimators')
find_df['xgb_model_abs_testing_MAE']=xgb_model_abs_testing
find_df['xgb_model_abs_training_MAE']=xgb_model_abs_training

find_df.plot()

stat_best_n_estimators=np.argmin(find_df['xgb_model_abs_testing_MAE'])
ch_n_estimators=stat_best_n_estimators

In [ ]:
xgb_model_abs_testing = list()
xgb_model_abs_training = list()
rng = np.arange(0.05, 0.65, 0.05)
for i in rng:
    xgb = xgboost.XGBRegressor(n_estimators=ch_n_estimators, random_state=17, learning_rate=i)
    xgb.fit(X_train, y_train)
    xgb.score(X_test, y_test)
    xgb_model_abs_testing.append(mean_absolute_error(xgb.predict(X_test), y_test))
    xgb_model_abs_training.append(mean_absolute_error(xgb.predict(X_train), y_train))
    
find_df=pd.DataFrame(rng, columns=['learning_rate']).set_index('learning_rate')
find_df['xgb_model_abs_testing_MAE']=xgb_model_abs_testing
find_df['xgb_model_abs_training_MAE']=xgb_model_abs_training

find_df.plot()

stat_best_learning_rate=np.argmin(find_df['xgb_model_abs_testing_MAE'])
ch_learning_rate=0.15

In [ ]:
xgb_model_abs_testing = list()
xgb_model_abs_training = list()
rng = np.arange(1, 11)
for i in rng:
    xgb = xgboost.XGBRegressor(n_estimators=ch_n_estimators, random_state=17, learning_rate=ch_learning_rate, max_depth=i)
    xgb.fit(X_train, y_train)
    xgb.score(X_test, y_test)
    xgb_model_abs_testing.append(mean_absolute_error(xgb.predict(X_test), y_test))
    xgb_model_abs_training.append(mean_absolute_error(xgb.predict(X_train), y_train))
    
find_df=pd.DataFrame(rng, columns=['max_depth']).set_index('max_depth')
find_df['xgb_model_abs_testing_MAE']=xgb_model_abs_testing
find_df['xgb_model_abs_training_MAE']=xgb_model_abs_training

find_df.plot()

stat_best_max_depth=np.argmin(find_df['xgb_model_abs_testing_MAE'])
ch_max_depth=2

In [ ]:
ch_max_depth=2

In [ ]:
import xgboost
from sklearn.metrics import mean_absolute_error, mean_squared_error
xgb = xgboost.XGBRegressor(n_estimators=ch_n_estimators, random_state=17, learning_rate=ch_learning_rate, max_depth=ch_max_depth)
xgb.fit(X_train, y_train)
print('R^2 train:', xgb.score(X_train, y_train))
print('R^2 test:', xgb.score(X_test, y_test))
print('Total: ', np.sum(y_test))
print('Total: ', np.sum(xgb.predict(X_test)))
print('MAE train', mean_absolute_error(xgb.predict(X_train), y_train))
print('MAE test', mean_absolute_error(xgb.predict(X_test), y_test))

In [ ]:
res_test=pd.DataFrame(X_test, columns=X.columns)
res_test['fact']=y_test
res_test['predict']=xgb.predict(X_test)


sh=res_test.sample(150).reset_index()
sh['ind']=sh.index

In [ ]:
check(sh, 'ind', 'fact', 'predict', 'gg')

In [ ]:
features_df = pd.DataFrame(data=xgb.feature_importances_.reshape(1, -1), columns=X.columns).sort_values(axis=1, by=[0], ascending=False).T

In [ ]:
features_df

In [ ]:
res_test.groupby('month').mean()[['fact']]

In [ ]:
res_test.groupby('hour').mean()[['fact']]